In [1]:
# !pip install --upgrade langchain accelerate transformers
# !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
!export PYTORCH_ENABLE_MPS_FALLBACK=1

In [2]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers import pipeline, AutoTokenizer

In [3]:
# Globals
repo_id = 'tiiuae/falcon-7b-instruct'

tokenizer = AutoTokenizer.from_pretrained(repo_id)
generate_text = pipeline(
    "text-generation",
    max_length=1000,
    model=repo_id,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    model_kwargs={
        'pad_token_id': tokenizer.eos_token_id
    }
)

transformer_pipeline = HuggingFacePipeline(pipeline=generate_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Response Schemas
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days did it take for the product to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any sentences about the value or price, and output them as a comma separated JSON array. Do not repeat sentences.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
# print(format_instructions)

In [5]:
template_string = """\
For the following text, extract the following information and present result as a JSON object following the schema enclosed by <>:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python array.

text: {text}

<{format_instructions}>
"""

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

prompt_template = PromptTemplate.from_template(template_string)

In [6]:
# LLM Chain
llm_chain = LLMChain(
    llm=transformer_pipeline,
    prompt=prompt_template
)

In [7]:
prediction = llm_chain.predict(
    format_instructions=format_instructions,
    text=customer_review
)

/Users/bsantanna/miniforge3/envs/ml/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [8]:
print(prediction)


The output should be:

```
{
    "gift": "True",
    "delivery_days": "2",
    "price_value": "True",
    "text": "This leaf blower is pretty amazing. It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [9]:
parsed_output = output_parser.parse(prediction)
print(type(parsed_output))

<class 'dict'>


In [10]:
print(f"Order delivery days: {parsed_output['delivery_days']}")
print(f"Order is a gift: {parsed_output['gift']}")

Order delivery days: 2
Order is a gift: True
